In [3]:
import pandas as pd

ds_1File = pd.read_excel ('germany-new-demo-current.xlsx')

In [4]:
#Making a copy so I dont need to read the entire dataset everytime I want to restart
ds_1 = ds_1File
ds_1

,Company Name 1,Company Name 2,Company Name 3,Street,Postal Code,City,Postfach,PLZ_Postfach,Anrede,Titel,...,Branchencode3,Branchencode4,Branchencode5,Branchencode6,Branchencode7,Branchencode8,Branchencode9,Branchencode10,Branchencode11,Branchencode12
0,12 Inch Records e.K.,NaN,NaN,Verlängerte Werderstr 40,12524,Berlin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12 ONE Media & More GmbH,NaN,NaN,Ludwig-Jahn-Str. 3,47533,Kleve,NaN,NaN,Herr,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12. Schule Konrad-Wachsmann-,Oberschule,NaN,Geithainer Str. 12,12627,Berlin,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1200-Jahre Esting e. V.,NaN,NaN,Edelweißweg 5,82140,Olching,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,122 Grundschule,Am Palitzschhof,NaN,Gamigstr. 30,1239,Dresden,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985071,NaN,Gründungsmanagem. und Gründ.,Did. mit beschränkter Haftung,Südblick 4,89075,Ulm,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
985072,NaN,NaN,NaN,Boschstr. 6,91183,Abenberg,NaN,NaN,Herr,NaN,...,Eisen- und Metallgießereien,"Steuer-, Mess- und Regeltechnik Hersteller",Computer und -Zubehör Einzelhandel,Industriebedarf und Betriebseinrichtungen Groß...,Gießereibedarf Großhandel,Industrieroboter Hersteller,NaN,NaN,NaN,NaN
985073,NaN,NaN,NaN,Gutendorf 14,93471,Arnbruck,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
985074,NaN,für die Region Schweinfurt -,NaN,Von-Münster-Str. 41,97440,Werneck,NaN,NaN,Herr,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Dropping unnecessary columns:
droppedcolumns = ["Company Name 2",'Company Name 3','Postfach','PLZ_Postfach','Anrede','Titel','PHONE','FAX',
                  'First Name','Last Name','Function','Abteilungszusatz','Anredezeile','Vorwahl',
                  'E-Mail Address','Website','Regierungsbezirk','Branchencode2','Branchencode3','Branchencode4', 'Branchencode5', 'Branchencode6', 
                  'Branchencode7', 'Branchencode8', 'Branchencode9', 'Branchencode10', 
                  'Branchencode11', 'Branchencode12']
ds_1 = ds_1.drop(droppedcolumns, axis = 1)
ds_1.columns

Index(['Company Name 1', 'Street', 'Postal Code', 'City', 'Bundesland',
       'Kreis', 'Beschaeftigtenklasse', 'Branchencode1'],
      dtype='object')

In [7]:
###Datenbereinigung

#Drop rows where Comapany Name is Nan
ds_1 = ds_1[ds_1['Company Name 1'].notna()]
#Drop rows where Comapany Name is True
ds_1 = ds_1[ds_1['Company Name 1'] != True]

In [7]:
# Filter out companys in a very small sector with only a few entries
#--> helps to filter out very specific, niche companys in small sectors that arent viable for clustering
Branchen = ds_1.value_counts('Branchencode1')

relevanteBranchen = Branchen[Branchen >1000] #1000 is a custom variable (interchangeable according to user preference)
relevanteBranchen = relevanteBranchen.index.tolist()
ds_1 = ds_1[ds_1['Branchencode1'].isin(relevanteBranchen)]

ds_1.value_counts('Branchencode1')

Branchencode1
Serviceunternehmen                              29284
Vereine                                         24271
Auto-Reparaturwerkstätten und -Einzelhändler    20729
Hotels                                          14790
Restaurants und Gaststätten                     12563
                                                ...  
Gärtnereien                                      1015
Katholische Pfarrämter                           1014
Reitsport-Vereine                                1010
Kraftfahrwesen-Sachverständige                   1010
Holz Großhandel                                  1008
Length: 202, dtype: int64

In [8]:
#Filtering out irrelevant sectors like schools, clubs, etc.
# --> dependenat on filtering value from above (this list: for value 1000)
irrelevanteBranchen = ['Vereine', 'Spiel- und Sport-Vereine','Ämter und Behörden', 
                       'Stadt- und Gemeindeverwaltungen','Erwachsenenbildung','Fahrschulen, gesamt',
                       'Kulturelle Einrichtungen','Schieß- und Schützen-Vereine','Musik-Vereine',
                       'Politische Vereine','Grund- und Hauptschulen','Fußball-Vereine','Turn-Vereine',
                       'Stadtverwaltungen','Verbände und Vereinigungen',
                       'Schulen, Erwachsenenbildung, Wissenschaft und Forschung',
                       'Kultur, Unterhaltung und Öffentliche Einrichtungen','Apparate Hersteller','Gymnasien',
                      'Evangelische Pfarrämter','Gemeindeverwaltungen und Gemeindeämter',
                       'Kindergärten, -Horte und -Tagesstätten','Fachschulen','Museen','Gesang-Vereine','Chöre',
                      'Ortschaftsverwaltungen','Anstalten für Weiterbildung','Tanzschulen','Tennis-Vereine',
                       'Kunstgalerien und Museen','Katholische Pfarrämter','Reitsport-Vereine',
                       'Allgemeinbildende Schulen','Interessenvertretungsvereine']

ds_1 = ds_1[~ds_1['Branchencode1'].isin(irrelevanteBranchen)]


In [9]:
##Creation of csv file for sector grouping task
#betrachteteBranchen = ds_1F.value_counts('Branchencode1') #maybe group these together
#betrachteteBranchenDf = pd.DataFrame(betrachteteBranchen)
#betrachteteBranchenDf.to_csv('betrachtete Branchen.csv',sep=";")#columns=['Branchencode1','O'])
#betrachteteBranchenDf

In [10]:
#################################
#Geting coordinates for postcodes
#################################

#Loading dataset with postcodes and coordinates of their geographical center
#From: https://github.com/WZBSocialScienceCenter/plz_geocoord/blob/master/plz_geocoord.csv
plz_coordinates = pd.read_csv('plz_geocoord.csv')

#Def function that looks up postcode in dataset df and returns: (latitude,longitude) as a tupel
def lookup(plz,df):
    try:
        lat,lng=(df[df['Unnamed: 0'] == plz]['lat'].iloc[0],df[df['Unnamed: 0'] == plz]['lng'].iloc[0])
        return (lat,lng)
    except IndexError:
        return (float('nan'),float('nan'))

In [11]:
#Applying lookup to all entries of the dataset, creating a new column
ds_1['PLZ_Coordinates'] = ds_1.apply(lambda x: lookup(x['Postal Code'],plz_coordinates), axis=1)
ds_1

,Company Name 1,Street,Postal Code,City,Bundesland,Kreis,Beschaeftigtenklasse,Branchencode1,PLZ_Coordinates
5,1-2-3 Abfluss-Frei GmbH,In den Hessengärten 38,61352,Bad Homburg,Hessen,Hochtaunuskreis,2011-06-10 00:00:00,Serviceunternehmen,"(50.2171728, 8.6540643)"
7,123 Braut e.K.,Heidelberger Str. 5-7,68519,Viernheim,Hessen,Bergstraße,bis 5,Damenbekleidung Einzelhandel,"(49.5574391, 8.5760911)"
8,123 CAD,Rauenthaler Str. 6,65197,Wiesbaden,Hessen,"Wiesbaden, Landeshauptstadt",bis 5,Softwarehäuser,"(50.0843808, 8.2080971)"
10,1-2-3 Gebäudemanagement,Fredericiastr. 28,14059,Berlin,Berlin,"Berlin, Stadt",bis 5,Serviceunternehmen,"(52.5212611, 13.2870699)"
11,1-2-3 Gebäudemanagement GmbH,Schützenstr. 107,22761,Hamburg,Hamburg,"Hamburg, Freie und Hansestadt",51-100,Gebäudereinigungsunternehmen,"(53.56785259999999, 9.9118726)"
...,...,...,...,...,...,...,...,...,...
985071,NaN,Südblick 4,89075,Ulm,Baden-Württemberg,"Ulm, Universitätsstadt",bis 5,Unternehmensberater,"(48.4170106, 10.0030932)"
985072,NaN,Boschstr. 6,91183,Abenberg,Bayern,Roth,bis 5,"Maschinen, Anlagen und Apparate Großhandel","(49.2415618, 10.9638001)"
985073,NaN,Gutendorf 14,93471,Arnbruck,Bayern,Regen,2011-06-10 00:00:00,"Radio, Funk, Fernseh, Phono und Video Einzelha...","(49.1303578, 12.9996546)"
985074,NaN,Von-Münster-Str. 41,97440,Werneck,Bayern,Schweinfurt,bis 5,Serviceunternehmen,"(49.9817885, 10.0842752)"


In [12]:
##############################
#Assigning custom sector names
##############################

df = ds_1 #Creating a copy so code can easily be restarted

#Creating lists for custom Branchencode (subdivision done by us)
Service = ['Serviceunternehmen',
           'Dienstleistung',
           'Friseursalons',
           'Technische Kundendienste und Reparaturwerkstätten',
           'Verleih, Vermittlung und Vermietung Dienstleistung',
           'Gebäudereinigungsunternehmen',
           'Pflegeheime',
           'Fotografen und Fotostudios',
           'Designer',
           'Reinigungsunternehmen',
           'Zeitarbeitsunternehmen',
           'Kosmetiksalons',
           'Grafiker',
           'Seminarveranstalter',
           'Privatzimmervermieter',
           'Autovermietung',
           'Partyservice',
           'Fotokopier- und Vervielfältigungsbetriebe',
           'Baumaschinenvermietungen']

Versicherung =['Versicherungsgesellschaften, Versicherungsagenturen, Krankenkassen und Krankenkassengeschäftsstellen',
               'Versicherungsagenturen und -Vermittler (zertifiziert)',
               'Versicherungsagenturen und -Vermittler',
               'Viktoria Versicherungen']

Immobilien = ['Immobilienmakler und Immobilienbüros']

Marketing = ['Werbeagenturen',
             'Marketingunternehmen',
             'Werbeunternehmen',
             'Public Relations-Agenturen']

Kultur_Sozial = ['Film, Funk und Fernsehen',
                 'Multimediaagenturen',
                 'Sozialwesen']

Vereine = ['Förderungs-Vereine',
           'Jugend-Vereine']

Verwaltung_Beratung = ['Steuerberatende Berufe',
                       'Unternehmensberater',
                       'Verwaltungsgesellschaften',
                       'Consultingunternehmen',
                       'Hausverwalter',
                       'Haus- und Gebäudeverwalter',
                       'Wirtschafts-, Steuer- und Unternehmensberatung',
                       'Sachverständige (selbständige)',
                       'Beteiligungsgesellschaften',
                       'Interessenvertretungsvereine',
                       'Holdinggesellschaften',
                       'Planungsbüros',
                       'Immobilienmakler',
                       'Pressewesen',
                       'Innungen',
                       'Kraftfahrwesen-Sachverständige']

IT = ['EDV-Dienstleistungsunternehmen',
      'Softwarehäuser',
      'Elektrotechnik, Elektronik und EDV Gesamt']

Logistik = ['Speditionen',
            'Versandhandel',
            'Transportunternehmen',
            'Lagerhäuser Dienstleistung']

Einzelhandel = ['Auto-Reparaturwerkstätten und -Einzelhändler',
                'Computer Einzelhandel',
                'Vertriebsunternehmen',
                'Bäckereien',
                'Bekleidung Einzelhandel',
                'Radio, Funk, Fernseh, Phono und Video Einzelhandel',
                'Elektrotechnik, Elektronik und EDV Einzelhandel',
                'Möbel Einzelhandel',
                'Sportartikel, Spielwaren und Musikinstrumente Einzelhandel',
                'Bauelemente, Baustoffe, Bau- und Heimwerkerbedarf Einzelhandel',
                'Lebensmittel Einzelhandel',
                'Metallwaren Einzelhandel',
                'Autozubehör Einzelhandel',
                'Getränke Einzelhandel',
                'Zweiräder Einzelhändler und Werkstätten',
                'Nahrungs-, Genussmittel und Getränke Einzelhandel',
                'Elektrotechnik und Elektronik Vertrieb',
                'Bücher Einzelhandel',
                'Fotoapparate und Fotobedarf Einzelhandel',
                'Konditoreien',
                'Sport- und Freizeitartikel Einzelhandel',
                'Metzgereien',
                'Fahrzeugteile und -Zubehör Einzelhändler und Werkstätten',
                'Schuhe Einzelhandel',
                'Damenbekleidung Einzelhandel',
                'Glas-, Keramik- und Porzellanwaren Einzelhandel',
                'Software Einzelhandel',
                'Textilwaren Einzelhandel',
                'Schmuck Einzelhandel',
                'Computerzubehör Einzelhandel',
                'Blumen Einzelhandel',
                'Nahrungs-, Genussmittel und Getränke Vertrieb',
                'Werbe-, Geschenkartikel und Andenken Einzelhandel',
                'Elektrische Geräte Einzelhandel',
                'Chemie Einzelhandel',
                'Gärtnereien']

Bau = ['Bauunternehmen ( Hoch- und Tiefbauunternehmen)',
       'Architekten und Architekturbüros',
       'Malerbetriebe',
       'Bauelemente, Baustoffe und Bauinstallationen Vertrieb',
       'Metallbau',
       'Sanitärinstallateure',
       'Dachdeckereien',
       'Schreinereien und Tischlereien',
       'Spezialbauunternehmen',
       'Ingenieurbüros für Bauwesen',
       'Landschaftsbauunternehmen',
       'Heizungs-, Klima- und Lüftungsinstallateure',
       'Elektroinstallateure',
       'Altbausanierungsunternehmen',
       'Klimaanlageninstallateure',
       'Baudienstleistungen, Sonstige',
       'Innenausbauunternehmen',
       'Garten- und Landschaftsbau',
       'Wohnbauunternehmen',
       'Bauplanung',
       'Bauhandwerk',
       'Architekten für Verwaltungsbauten',
       'Fliesenleger',
       'Klempnereien und Installationsgeschäfte',
       'Architekten, Bauplanung und Baudienstleistung']

Großhandel = ['Großhandel',
              'Bauelemente, Baustoffe, Bauinstallationen Großhandel, Vertrieb, Ex- und Import',
              'Elektronik, Elektrotechnik und EDV Großhandel, Vertrieb, Ex-und Import',
              'Bauelemente und Bauausstattung Großhandel',
              'Medizinische Geräte und Instrumente Großhandel',
              'Holz Großhandel']

Gesundheit_Fitness = ['Apotheken, öffentliche',
                      'Ärzte, Gesundheitswesen und Sozialwesen',
                      'Ärzte und Heilberufe',
                      'Heilpraktiker',
                      'Zahnärzte',
                      'Fitness-Center',
                      'Physiotherapeuten',
                      'Apotheken',
                      'Internisten',
                      'Optiker',
                      'Gesundheitswesen',
                      'Zahntechnische Laboratorien, Dentallabors',
                      'Psychotherapeuten',
                      'Geburtshilfeärzte',
                      'Tierärzte',
                      'Orthopäden']

Finanzen =['Sparkassen',
           'Volksbanken, Raiffeisen- und Genossenschaftsbanken',
           'Finanz- und Anlageberater',
           'Banken',
           'Bausparkassen und Bausparkassenagenturen']

Tourismus_Gastro = ['Hotels',
                    'Restaurants und Gaststätten',
                    'Reisebüros',
                    'Hotelgewerbe und Gastronomie',
                    'Pensionen',
                    'Reiseveranstalter',
                    'Gasthöfe']

Maschinen_Ingenieure = ['Ingenieurbüros',
                        'Maschinen, Anlagen und Apparate Großhandel, Vertrieb, Ex- und Import',
                        'Maschinen, Anlagen und Apparate Großhandel']

Produzente_Hersteller_Industrie = ['Verlage',
                                   'Flaschnereien',
                                   'Kunststoff und Gummi Hersteller',
                                   'Offsetdruckereien',
                                   'Energiegewinnung, Energieversorgung und Wasserwirtschaft',
                                   'Druckindustrie Hersteller',
                                   'Bauelemente, Baustoffe und Bauinstallationen Hersteller',
                                   'Werkzeugbau und Werkzeuge Hersteller',
                                   'Elektrotechnik, Elektronik und EDV Hersteller',
                                   'Weinbauunternehmen und Weingüter',
                                   'Steuer-, Mess- und Regeltechnik Hersteller']

Handwerk = ['Eisen und Metall Handwerk']

Selbstständige = ['Selbständige Berufsgruppen, freie Berufe',
                  'Selbständige Berufsgruppen aus Kunst, Kultur und Sport']

Wissenschaft_Forschung = ['Wissenschaft und Forschung']



Sectors = [Service, 
           Versicherung, 
           Immobilien, 
           Marketing, 
           Kultur_Sozial,
           Vereine,
           Verwaltung_Beratung,
           IT,
           Logistik,
           Einzelhandel,
           Bau,
           Großhandel,
           Gesundheit_Fitness,
           Finanzen,
           Tourismus_Gastro,
           Maschinen_Ingenieure,
           Produzente_Hersteller_Industrie,
           Handwerk,
           Selbstständige,
           Wissenschaft_Forschung]

sectorDic = {
    "Service":Service,
    'Versicherung':Versicherung,
    'Immobilien':Immobilien,
    'Marketing':Marketing,
    'Kultur_Sozial':Kultur_Sozial,
    'Vereine':Vereine,
    'Verwaltung_Beratung':Verwaltung_Beratung,
    'IT':IT,
    'Logistik':Logistik,
    'Einzelhandel':Einzelhandel,
    'Bau':Bau,
    'Grosshandel':Großhandel,
    'Gesundheit_Fitness':Gesundheit_Fitness,
    'Finanzen':Finanzen,
    'Tourismus_Gastro':Tourismus_Gastro,
    'Maschinen_Ingenieure':Maschinen_Ingenieure,
    'Produzente_Hersteller_Industrie':Produzente_Hersteller_Industrie,
    'Handwerk':Handwerk,
    'Selbststaendige':Selbstständige,
    'Wissenschaft_Forschung':Wissenschaft_Forschung
}

In [13]:
##Def function that takes name of a sector according to the dataset and returns the respective, more general custom sector name (based on sectorDic)
import numpy as np

def getCustomLabel(a):
    
    for key,value in sectorDic.items():
        if a in value:
            return key
        
    return np.nan

In [14]:
#Applying getCustomLabel to every Entry in the dataset, creating a new column
df['CustomSector'] = df.apply(lambda x: getCustomLabel(x['Branchencode1']),axis=1)

In [15]:
#Storing the preprocessed data in an Excel File
df.to_excel('Preprocessed_Data_V2.xlsx')
df

,Company Name 1,Street,Postal Code,City,Bundesland,Kreis,Beschaeftigtenklasse,Branchencode1,PLZ_Coordinates,CustomSector
5,1-2-3 Abfluss-Frei GmbH,In den Hessengärten 38,61352,Bad Homburg,Hessen,Hochtaunuskreis,2011-06-10 00:00:00,Serviceunternehmen,"(50.2171728, 8.6540643)",Service
7,123 Braut e.K.,Heidelberger Str. 5-7,68519,Viernheim,Hessen,Bergstraße,bis 5,Damenbekleidung Einzelhandel,"(49.5574391, 8.5760911)",Einzelhandel
8,123 CAD,Rauenthaler Str. 6,65197,Wiesbaden,Hessen,"Wiesbaden, Landeshauptstadt",bis 5,Softwarehäuser,"(50.0843808, 8.2080971)",IT
10,1-2-3 Gebäudemanagement,Fredericiastr. 28,14059,Berlin,Berlin,"Berlin, Stadt",bis 5,Serviceunternehmen,"(52.5212611, 13.2870699)",Service
11,1-2-3 Gebäudemanagement GmbH,Schützenstr. 107,22761,Hamburg,Hamburg,"Hamburg, Freie und Hansestadt",51-100,Gebäudereinigungsunternehmen,"(53.56785259999999, 9.9118726)",Service
...,...,...,...,...,...,...,...,...,...,...
985071,NaN,Südblick 4,89075,Ulm,Baden-Württemberg,"Ulm, Universitätsstadt",bis 5,Unternehmensberater,"(48.4170106, 10.0030932)",Verwaltung_Beratung
985072,NaN,Boschstr. 6,91183,Abenberg,Bayern,Roth,bis 5,"Maschinen, Anlagen und Apparate Großhandel","(49.2415618, 10.9638001)",Maschinen_Ingenieure
985073,NaN,Gutendorf 14,93471,Arnbruck,Bayern,Regen,2011-06-10 00:00:00,"Radio, Funk, Fernseh, Phono und Video Einzelha...","(49.1303578, 12.9996546)",Einzelhandel
985074,NaN,Von-Münster-Str. 41,97440,Werneck,Bayern,Schweinfurt,bis 5,Serviceunternehmen,"(49.9817885, 10.0842752)",Service
